In [3]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from config.config import *
from libs.common import *

c:\Users\PC\Data\ads_fraud_detection


In [4]:
import tensorflow.keras as keras
import numpy as np
import pandas as pd 
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Conv1D, Attention, LSTM, Embedding, Bidirectional
import tensorflow as tf
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, Dense, Dropout, 
                                   LayerNormalization, GlobalAveragePooling1D)
from tensorflow.keras.models import Model
from tabnet import TabNet
# from tabnet import TabNetClassifier
import tabnet

In [5]:
save_dir=f"{exps_dir}/exp1/exp_base"
if os.path.exists(save_dir) == False: 
  os.makedirs(save_dir, exist_ok=True)



test_size=0.33
seed=42

In [6]:
# feature_selection=dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']
# feature_selection = {key: value for key, value in feature_selection.item().items()}
# feature_selection

* kiểm tra và tạo các thư mục (nếu chưa có)
* tập test 33%

# 5. Xây dựng và đánh giá mô hình học sâu Neutual Network

In [7]:
# !python -m pip install tabnet


In [8]:
# bg_model=joblib.load(f'{save_dir}/bg_model.joblib')
# rf_model=joblib.load(f'{save_dir}/rf_model.joblib')
# svm_model=joblib.load(f'{save_dir}/svm_model.joblib')

In [9]:
data_train=pd.read_excel(f'{save_dir}/datatrain.xlsx')
data_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
0,Y,Middle,IL,250/500,1000,817.28,0,MALE,High School,sales,cross-fit,unmarried,62200,0,Multi-vehicle Collision,Rear Collision,Total Loss,Ambulance,SC,Columbus,Evening,3,?,1,1,NO,60190,4630,9260,46300,Mercedes,ML350,2018++,Y,21,6,1994,3,1,2015
1,N,Young,IN,100/300,2000,765.64,0,MALE,MD,sales,base-jumping,not-in-family,0,-22200,Multi-vehicle Collision,Front Collision,Major Damage,Ambulance,SC,Northbrook,Morning,3,?,0,3,?,31350,2850,5700,22800,Honda,Accord,1995-20001,Y,11,6,2008,16,2,2015
2,Y,Middle,OH,100/300,2000,1422.21,0,FEMALE,Associate,handlers-cleaners,polo,unmarried,0,-46400,Single Vehicle Collision,Side Collision,Major Damage,Ambulance,WV,Riverwood,Night,1,YES,2,1,NO,85900,17180,17180,51540,Suburu,Forrestor,2002-2008,Y,29,7,1996,16,1,2015
3,Y,Middle,IL,100/300,2000,1133.27,0,FEMALE,High School,craft-repair,chess,wife,0,-56400,Multi-vehicle Collision,Side Collision,Total Loss,Other,NY,Northbrook,Night,2,?,0,2,YES,60500,11000,5500,44000,Ford,F150,1995-20001,Y,28,5,2014,1,3,2015
4,Y,Middle,OH,250/500,1000,1406.91,0,MALE,MD,craft-repair,sleeping,husband,53300,0,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,Morning,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2002-2008,Y,17,10,2014,25,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,Y,Old,OH,500/1000,1000,1125.37,0,FEMALE,Associate,prof-specialty,basketball,husband,0,-56400,Multi-vehicle Collision,Rear Collision,Major Damage,Police,VA,Northbend,Morning,3,?,0,2,YES,79800,6650,19950,53200,Saab,95,1995-20001,Y,7,10,1990,13,1,2015
666,Y,Old,OH,250/500,2000,1589.54,0,MALE,College,handlers-cleaners,camping,husband,55400,0,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,VA,Arlington,Morning,3,?,2,0,YES,85300,17060,8530,59710,Toyota,Highlander,2002-2008,N,15,4,2013,27,1,2015
667,Y,Middle,IN,100/300,1000,1023.11,0,FEMALE,MD,adm-clerical,golf,own-child,0,-45300,Multi-vehicle Collision,Rear Collision,Minor Damage,Other,NY,Northbend,Evening,3,NO,1,2,YES,58850,10700,10700,37450,Accura,MDX,1995-20001,N,16,11,2004,6,2,2015
668,N,Young,OH,250/500,500,1075.41,0,MALE,MD,machine-op-inspct,reading,wife,55200,0,Single Vehicle Collision,Side Collision,Total Loss,Police,PA,Northbend,Morning,1,NO,1,0,NO,73400,7340,7340,58720,Dodge,Neon,1995-20001,N,29,8,1999,15,2,2015


In [10]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
class_weights_dict=dict(np.load(f'{save_dir}/class_weights_dict.npz',allow_pickle=True))['class_weights_dict']
class_weights_dict = {key: value for key, value in class_weights_dict.item().items()}
class_weights_dict

{0: 0.6555772994129159, 1: 2.106918238993711}

In [11]:

data_train = pd.concat([x_train, y_train], axis=1)

data_test = pd.concat([x_test, y_test], axis=1)
data_test.head()

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date,fraud_reported
0,0,2,0,1,2,-0.513764,1,0,6,4,16,1,0.314681,0.921115,2,2,2,0,6,6,2,0,2,1,3,0,1.406346,1.857804,1.870185,1.013395,1,5,1,1.438991,0.161918,1.591092,1.143381,-0.948098,0,0
1,1,2,0,2,1,0.690401,1,0,2,3,8,0,1.445797,0.921115,0,1,2,0,0,5,0,2,0,2,3,2,0.067678,-0.364631,-0.395315,0.285379,9,29,1,-0.768072,-1.251652,0.927802,0.573418,-0.948098,0,0
2,1,1,1,1,1,-1.190728,1,0,4,2,15,0,-0.869045,-1.225352,0,2,1,2,6,5,0,2,0,1,3,0,0.628774,1.340478,-0.083123,0.552792,11,21,1,0.625863,0.161918,-1.592700,0.345432,0.983213,0,0
3,1,1,1,0,2,0.481517,1,1,1,0,17,4,0.735562,-1.362585,0,1,2,3,4,4,0,2,1,0,0,1,0.446797,-0.157700,1.140331,0.375921,7,36,2,-0.651911,1.575488,-1.327384,-0.110538,0.983213,0,0
4,0,2,2,1,1,-0.597410,1,0,2,6,18,1,-0.869045,-0.131006,1,0,3,3,4,0,1,0,2,2,0,2,-1.675509,-1.328927,-1.247515,-1.677052,2,0,1,-0.651911,-0.686224,0.131854,-0.338524,0.983213,0,0


In [12]:


# Dữ liệu label đã được mã hóa
def oneHot(arr):
    labels = np.array(arr)
    encoder = OneHotEncoder(sparse=False)
    
    labels_reshaped = labels.reshape(-1, 1)
    
    encoder.fit(labels_reshaped)
    
    onehot_labels = encoder.transform(labels_reshaped)
    # onehot_labels=pd.DataFrame(onehot_labels)
    return onehot_labels

y_train_onehot=oneHot(y_train)
y_test_onehot=oneHot(y_test)
print(y_test_onehot.shape,y_train_onehot.shape)


(330, 2) (670, 2)


In [13]:
y_test_onehot.reshape(-1, 1)
print(y_test_onehot.shape)

(330, 2)


In [14]:
def f1_score(y_true, y_pred):
    """
    Macro F1 score metric.
    """
    y_pred = K.round(y_pred)    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    false_positives = K.sum(K.round(K.clip(y_pred - y_true, 0, 1)), axis=0)
    false_negatives = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=0)    
    precision = true_positives / (true_positives + false_positives + K.epsilon())
    recall = true_positives / (true_positives + false_negatives + K.epsilon())    
    f1_scores = 2 * (precision * recall) / (precision + recall + K.epsilon())    
    macro_f1_score = K.mean(f1_scores)
    
    return macro_f1_score

#### * Xây dựng model:

#### * Xây dựng Stacked_model:

In [15]:
def f1_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    # Calculate true positives, predicted positives, and possible positives for each class
    tp = tf.reduce_sum(y_true * y_pred, axis=0)
    predicted_positives = tf.reduce_sum(y_pred, axis=0)
    possible_positives = tf.reduce_sum(y_true, axis=0)
    
    # Calculate precision and recall for each class
    precision = tp / (predicted_positives + tf.keras.backend.epsilon())
    recall = tp / (possible_positives + tf.keras.backend.epsilon())
    
    # Calculate F1 score for each class
    f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    
    # Calculate the macro F1 score
    f1_macro = tf.reduce_mean(f1)
    
    # Return the F1 loss
    return 1 - f1_macro

In [16]:
# Create sample DataFrame
n_samples = 1000
n_features = 10

# Generate random data
df = data_train


In [17]:
def prepare_tabnet_data(df, feature_dim=1024, output_dim=512):
    """
    Prepare data for TabNet model from DataFrame
    """
    # 1. Create numeric feature columns for each input feature
    feature_columns = []
    num_features = len(df.columns)
    for col in df.columns:
        # Create numeric column with proper shape
        feature_columns.append(
            tf.feature_column.numeric_column(
                key=col,
                shape=(1,),
                dtype=tf.float32
            )
        )
    
    # 2. Initialize TabNet model
    model = TabNet(
        feature_columns=feature_columns,
        num_features=num_features,
        feature_dim=feature_dim,       # Na parameter
        output_dim=output_dim,         # Nd parameter
        # num_decision_steps=5,          # N_steps parameter
        relaxation_factor=1.5,         # gamma parameter
        sparsity_coefficient=1e-5,     # lambda_sparse parameter
        norm_type='group',             # Using group normalization
        batch_momentum=0.98,
        virtual_batch_size=128,
        num_groups=2,
        epsilon=1e-5
    )
    
    # 3. Format input dictionary
    input_dict = {}
    for col in df.columns:
        # Each column becomes a key in the dictionary
        input_dict[col] = df[col].values.reshape(-1, 1).astype(np.float32)
    
    return model, input_dict

# Usage example:
# For a batch of data
batch_size = 32
df_batch = df.iloc[:batch_size]

model, input_dict = prepare_tabnet_data(
    df=df,
    feature_dim=1024,
    output_dim=512
)

# Get TabNet output
output = model(input_dict)
print(f"Output shape: {output.shape}")  # Should be (batch_size, output_dim)


Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


TypeError: __init__() got an unexpected keyword argument 'num_decision_steps'